In [1]:
# Imports
from typing import Union
import pandas as pd
import requests
from datetime import datetime
import pytse_client as tse
import fetch
import config
from utils import debug_log, data_log, ar_to_fa_series, ar_to_fa, fa_to_ar, fa_to_ar_series, flatten_json

pd.set_option('display.max_columns', 30) # pd.options.display.max_columns = 3
pd.set_option('display.max_rows', 7) # pd.options.display.max_rows = 3
pd.set_option('display.float_format', '{:.2f}'.format) # = pd.options.display.float_format = '{:.0f}'.format


#### Common Parameters

In [ ]:
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
    "Referer": "http://main.tsetmc.com/StaticContent/WebServiceHelp",
}
cookie_jar = {"ASP.NET_SessionId": "wa40en1alwxzjnqehjntrv5j"}
test_ids = ['46348559193224090','20024911381434086','22129017544200','94378630649293']
# currentDateTime = datetime.now().strftime("%Y-%m-%d %H-%M-%S %p")

last_date = fetch.api_last_possible_deven()
instruments_df = fetch.get_instruments()
identities_df = fetch.get_identities()

insCodes = list(instruments_df.index)
insCodes_indices_excluded = list(instruments_df[instruments_df['cComVal'] != '6'].index)

print(f'Last Date: {last_date}')
print('Instruments: ', len(insCodes))
print('Instruments (Excluding Indices): ', len(insCodes_indices_excluded))
print('Indices: ', len(insCodes) - len(insCodes_indices_excluded))

In [ ]:
## Getting Prices
daily_prices = fetch.get_quotes(insCodes)
print(f'{len(daily_prices.keys())} Tickers have history.')

AttributeError: module 'fetch' has no attribute 'load_quotes'

## Categorization

In [4]:
def filter_tickers_by_industry(cSecVal: int) -> list:
    return list(identities_df[identities_df['sector_cSecVal'] == cSecVal].index)

In [5]:
khodroi = filter_tickers_by_industry(34)

## Volume Filter

In [8]:

def volume_filter(insCodes: list) -> list:
    
    daily_prices = fetch.get_quotes(insCodes)
    
    list_of_dfs = []
    ticker_names= []
    # TODO: درست کردن فیلتر «نمادهای معامله شده امروز/روز آخر کاری»ا
    for insCode in insCodes:
        last_row = daily_prices[insCode].iloc[-1]
        if (
            daily_prices[insCode].index[-1] == int(last_date) and
            last_row['<VOL>'] > last_row['<VOL> MA 3'] and 
            last_row['<VOL>'] > last_row['<VOL> MA 5'] and
            last_row['<VOL>'] > last_row['<VOL> MA 9'] and
            last_row['<VOL>'] > last_row['<VOL> MA 26'] and
            last_row['<VOL>'] > last_row['<VOL> MA 52'] and
            last_row['<VOL>'] > last_row['<VOL> MA 90']
        ):
            list_of_dfs.append(daily_prices[insCode])
            ticker_names.append(insCode)
            debug_log.info(f'{insCode} Checked.')
    print(len(ticker_names),'\n')
    return ticker_names


In [ ]:
volume_filter(list(daily_prices.keys()))